In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install plantcv
from plantcv import plantcv as pcv
import numpy as np
from tqdm.notebook import tqdm
# pcv.params.debug = "plot"
%matplotlib inline

     |████████████████████████████████| 230 kB 32.7 MB/s 
     |████████████████████████████████| 40 kB 4.8 MB/s 
     |████████████████████████████████| 58.0 MB 1.3 MB/s 
     |████████████████████████████████| 802 kB 58.0 MB/s 
     |████████████████████████████████| 1.0 MB 57.9 MB/s 
     |████████████████████████████████| 132 kB 67.3 MB/s 
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Attempting uninstall: dask
    Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
  Attempting uninstall: distributed
    Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninst

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import re
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from tqdm import tqdm

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [4]:
def get_area(img_binary):
    return np.round(img_binary[img_binary==255].size/img_binary.size, 4)

In [5]:
path = '/content/drive/MyDrive/growth/open/'
train = pd.read_csv(path + 'train_set.csv')
test = pd.read_csv(path + 'test_set.csv')

In [6]:
train['species'].unique()

array(['bc', 'lt'], dtype=object)

In [7]:
after_score = []
for i in tqdm(range(len(test))):
    img, path, filename = pcv.readimage(test['after_file_path'][i])
    a = pcv.rgb2gray_lab(img, 'a')
    img_binary = pcv.threshold.binary(a, 115, 255, 'dark')
    filtered_img = pcv.erode(img_binary, 5, 3)
    dilated = pcv.dilate(filtered_img, 5, 3)
    b = get_area(dilated)
    after_score.append(b)

100%|██████████| 16653/16653 [1:52:06<00:00,  2.48it/s]


In [8]:
before_score = []
for i in tqdm(range(len(test))):
    img, path, filename = pcv.readimage(test['before_file_path'][i])
    a = pcv.rgb2gray_lab(img, 'a')
    img_binary = pcv.threshold.binary(a, 115, 255, 'dark')
    filtered_img = pcv.erode(img_binary, 5, 3)
    dilated = pcv.dilate(filtered_img, 5, 3)
    b = get_area(dilated)
    before_score.append(b)

100%|██████████| 16653/16653 [1:46:08<00:00,  2.61it/s]


In [9]:
test['before_score'] = before_score
test['after_score'] = after_score
test['diff_score'] = test['after_score'] - test['before_score']

In [11]:
test.to_csv('/content/drive/MyDrive/growth/open/test_score.csv', index=False)